# Credit Risk Scorecard Demo

## Before you Begin
To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also create a documentation project on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the Client Library

In [ ]:
# %pip install --upgrade validmind

## Initialize the Client Library
In a browser, go to the Client Integration page of your documentation project and click Copy to clipboard next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

This step requires a documentation project. Learn how you can create one.

Next, replace this placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "clk00h0u800x9qjy67gduf5om"
)

## Use Case

#### Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, which translates into an estimated Point-in-Time (PiT) PD. The PiT PD reflects the borrower's likelihood of default at a specific point in time, accounting for both current and foreseeable future conditions.

Additionally, for a holistic view of credit risk, it's essential to estimate the Lifetime PD. The Lifetime PD, as the name suggests, predicts the borrower's likelihood of default throughout the life of the exposure, taking into account potential future changes in the economic and financial conditions.

#### Import Libraries

In [ ]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

# Standard library imports
import re
import pickle
from datetime import datetime
from typing import List

# Data handling and analysis imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import statsmodels.api as sm
import inspect

# Visualization imports
%matplotlib inline

# Scorecard development
import scorecardpy as sc

#### Processing Functions

In [ ]:
unused_variables = [
    "id", "member_id", "funded_amnt", "emp_title", "url", "desc", "application_type",
    "title", "zip_code", "delinq_2yrs", "mths_since_last_delinq", "mths_since_last_record",
    "revol_bal", "total_rec_prncp", "total_rec_late_fee", "recoveries", "out_prncp_inv", "out_prncp", 
    "collection_recovery_fee", "next_pymnt_d", "initial_list_status", "pub_rec",
    "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "pymnt_plan",
    "tot_coll_amt", "tot_cur_bal", "total_rev_hi_lim", "last_pymnt_d", "last_credit_pull_d",
    'earliest_cr_line', 'issue_d']

In [ ]:
def save_model(model, df, base_filename):
    """Save a model and a dataframe with a timestamp in the filename"""
    # Get current date and time
    now = datetime.datetime.now()

    # Convert the current date and time to string
    timestamp_str = now.strftime("%Y%m%d_%H%M%S")

    filename = f'{base_filename}_{timestamp_str}.pkl'

    # Save the model and dataframe
    with open(filename, 'wb') as file:
        pickle.dump((model, df), file)
        
    print(f"Model and dataframe saved as {filename}")

def get_numerical_columns(df):
        numerical_columns = df.select_dtypes(
            include=["int", "float", "uint"]
        ).columns.tolist()
        return numerical_columns

def get_categorical_columns(df):
        categorical_columns = df.select_dtypes(
            include=["object", "category"]
        ).columns.tolist()
        return categorical_columns

def add_target_column(df, target_column):
    # Assuming the column name is 'loan_status'
    df[target_column] = df['loan_status'].apply(lambda x: 0 if x == "Fully Paid" else 1 if x == "Charged Off" else np.nan)
    # Remove rows where the target column is NaN
    df = df.dropna(subset=[target_column])
    # Convert target column to integer
    df[target_column] = df[target_column].astype(int)
    return df

def variables_with_min_missing(df, min_missing_percentage):
    # Calculate the percentage of missing values in each column
    missing_percentages = df.isnull().mean() * 100

    # Get the variables where the percentage of missing values is greater than the specified minimum
    variables_to_drop = missing_percentages[missing_percentages > min_missing_percentage].index.tolist()

    # Also add any columns where all values are missing
    variables_to_drop.extend(df.columns[df.isnull().all()].tolist())

    # Remove duplicates (if any)
    variables_to_drop = list(set(variables_to_drop))

    return variables_to_drop

def clean_term_column(df, column):
    """
    Function to remove 'months' string from the 'term' column and convert it to categorical
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].str.replace(' months', '')
    
    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_emp_length_column(df, column):
    """
    Function to clean 'emp_length' column and convert it to categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].replace('n/a', np.nan)
    df[column] = df[column].str.replace('< 1 year', str(0))
    df[column] = df[column].apply(lambda x: re.sub('\D', '', str(x)))
    df[column].fillna(value = 0, inplace=True)

    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_inq_last_6mths(df, column):
    """
    Function to convert 'inq_last_6mths' column into categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")

    # Convert to categorical
    df[column] = df[column].astype('category')

def compute_outliers(series, threshold=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

def remove_iqr_outliers(df, target_column, threshold=1.5):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols.remove(target_column)  # Exclude target_column from numerical columns
    for col in num_cols:
        outliers = compute_outliers(df[col], threshold)
        df = df[~df[col].isin(outliers)]
    return df

def transform_woe_df(woe_df):
    # Select and rename columns
    transformed_df = woe_df[['variable', 'bin', 'count', 'count_distr', 'good', 'bad', 'badprob', 'woe', 'bin_iv', 'total_iv']].copy()
    transformed_df.rename(columns={
        'bin_iv': 'total_iv'
    }, inplace=True)
    
    # Create 'is_special_values' column (assuming there are no special values)
    transformed_df['is_special_values'] = False
    
    # Transform 'bin' column into interval format and store it in 'breaks' column
    transformed_df['breaks'] = transformed_df['bin'].apply(lambda x: '[-inf, %s)' % x if isinstance(x, float) else '[%s, inf)' % x)
    
    # Group by 'variable' to create bins dictionary
    bins = {}
    for variable, group in transformed_df.groupby('variable'):
        bins[variable] = group
    
    return bins

#### Developer Tasks

In [ ]:
# Import Raw Data
def import_raw_data(source): 
    print("Importing raw data from:", source)
    df_out = pd.read_csv(source)
    print(f"Data imported successfully with {df_out.shape[0]} rows and {df_out.shape[1]} columns.")
    return df_out

def add_target_column(df, target_column):
    # Check if 'loan_status' is in the DataFrame
    if 'loan_status' not in df.columns:
        raise ValueError("'loan_status' column not found in the DataFrame.")

    print("Converting 'loan_status' to target column...")
    # Assuming the column name is 'loan_status'
    df[target_column] = df['loan_status'].apply(lambda x: 0 if x == "Fully Paid" else 1 if x == "Charged Off" else np.nan)

    initial_row_count = df.shape[0]
    # Remove rows where the target column is NaN
    df = df.dropna(subset=[target_column])
    removed_rows = initial_row_count - df.shape[0]
    print(f"Removed {removed_rows} rows with undefined 'loan_status' values.")

    # Convert target column to integer
    df[target_column] = df[target_column].astype(int)
    print(f"Converted 'loan_status' to '{target_column}' and set its data type to integer.")
    
    return df


def variables_with_min_missing(df, min_missing_count):
    cols_to_drop = [col for col in df.columns if df[col].isnull().sum() / len(df) > min_missing_count / 100]
    return cols_to_drop

# Data Preparation: Add Definition of Default
def add_default_definition(df, default_column, unused_variables):
    print("Adding default definition...")

    df_out = df.copy()
    df_out = add_target_column(df_out, default_column)
    print("Added target column based on default column.")

    # Drop 'loan_status' variable 
    df_out.drop(columns='loan_status', axis=1, inplace=True)
    print("Dropped 'loan_status' column.")

    # Remove unused variables
    df_out = df_out.drop(columns=unused_variables)
    print(f"Dropped {len(unused_variables)} unused columns.")

    # Remove missing values
    min_missing_count = 80
    variables_to_drop = variables_with_min_missing(df_out, min_missing_count)
    df_out.drop(columns=variables_to_drop, axis=1, inplace=True)
    print(f"Dropped columns with more than {min_missing_count}% missing values.")
    df_out.dropna(axis=0, subset=["emp_length"], inplace=True)
    print("Dropped rows with missing 'emp_length'.")
    df_out.dropna(axis=0, subset=["revol_util"], inplace=True)
    print("Dropped rows with missing 'revol_util'.")

    return df_out

# Data Sampling: Data Split 
def data_split(df, target_column):
    df_out = df.copy()

    # Split data into train and test 
    X = df_out.drop(target_column, axis=1)
    y = df_out[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print(f"Training data has {X_train.shape[0]} rows and {X_train.shape[1]} columns.")
    print(f"Test data has {X_test.shape[0]} rows and {X_test.shape[1]} columns.")

    # Concatenate X_train with y_train to form df_train
    df_train = pd.concat([X_train, y_train], axis=1)

    # Concatenate X_test with y_test to form df_test
    df_test = pd.concat([X_test, y_test], axis=1)

    return df_train, df_test


# EDA: Drop Unused Categories
def drop_categories(df):
    df_out = df.copy()

    # Initial count
    initial_count = df_out.shape[0]

    # Select rows where purpose is 'debt_consolidation' or 'credit_card'
    df_out = df_out[df_out['purpose'].isin(['debt_consolidation', 'credit_card'])]
    print(f"Rows retained with purpose 'debt_consolidation' or 'credit_card': {df_out.shape[0]}")

    # Remove rows where grade is 'F' or 'G'
    df_out = df_out[~df_out['grade'].isin(['F', 'G'])]
    print(f"Rows after removing grades 'F' or 'G': {df_out.shape[0]}")

    # Remove rows where sub_grade starts with 'F' or 'G'
    df_out = df_out[~df_out['sub_grade'].str.startswith(('F', 'G'))]
    print(f"Rows after removing sub_grades starting with 'F' or 'G': {df_out.shape[0]}")

    # Remove rows where home_ownership is 'OTHER', 'NONE', or 'ANY'
    df_out = df_out[~df_out['home_ownership'].isin(['OTHER', 'NONE', 'ANY'])]
    print(f"Rows after removing home_ownership values 'OTHER', 'NONE', or 'ANY': {df_out.shape[0]}")

    print(f"Total rows dropped: {initial_count - df_out.shape[0]}")

    return df_out


# EDA: Drop Unused Features
def drop_features(df, to_drop):
    df_out = df.copy()

    # Before dropping
    initial_cols = df_out.shape[1]
    
    df_out.drop(columns=to_drop, axis=1, inplace=True)

    # After dropping
    after_drop_cols = df_out.shape[1]
    
    print(f"Dropped {initial_cols - after_drop_cols} columns.")
    print(f"Columns remaining after dropping: {after_drop_cols}")

    return df_out 

# EDA: Convert Features to WoE Values
def convert_to_woe(df, woe_df, target_col):
    df_out = df.copy()
    
    # Placeholder for the transformation function - you'll need to define or import it
    bins = transform_woe_df(woe_df)
    
    # Print how many features are getting transformed
    print(f"Converting {len(bins)} features to WoE values.")
    
    # Make sure we don't transform the target column
    if target_col in bins:
        del bins[target_col]
        print(f"Excluded {target_col} from WoE transformation.")
    
    # Apply the WoE transformation
    df_out = sc.woebin_ply(df_out, bins=bins)
    
    print(f"Successfully converted features to WoE values.")

    return df_out

# Model Training: Add Constant
def add_constant(df):
    df_out = df.copy()

    # Before adding constant
    initial_cols = df_out.shape[1]

    # Add constant
    df_out = sm.add_constant(df_out)

    # After adding constant
    after_add_cols = df_out.shape[1]

    print(f"Added constant to dataframe. Number of columns went from {initial_cols} to {after_add_cols}.")
    
    return df_out

# Model Training: Train Model
def train_model(df, target_column):
    
    # Ensure that the target column is in the DataFrame
    if target_column not in df.columns:
        raise ValueError(f"'{target_column}' not found in DataFrame.")

    # Get X (features) and y (target) from df
    X = df.drop(target_column, axis=1)  # Drop the target column to get features
    y = df[target_column]

    # Define the model
    model = sm.GLM(y, X, family=sm.families.Binomial())

    print(f"Training the model with {X.shape[1]} features and {X.shape[0]} data points.")

    # Fit the model
    model_fit = model.fit()

    print("Model trained successfully.")

    return model_fit



#### Developer Class

In [ ]:
import datetime
import inspect
import re
import pandas as pd
import logging

# Set up the logging configuration
logging.basicConfig(level=logging.INFO, format='INFO: %(message)s')

class Developer:
    def __init__(self):
        self.tasks_log = []
        self.tasks_details = []
        self.tasks = {}  # Dictionary to store tasks

    def add_task(self, task_id, task):
        """Register a task."""
        if task_id in self.tasks:
            raise ValueError(f"Task ID '{task_id}' already exists!")
        self.tasks[task_id] = {'task': task}
        return task_id

    def get_caller_info(self, frame):
        """Fetch the calling line of code and the variable names."""
        code_context = inspect.getframeinfo(frame).code_context
        line_of_code = code_context[0].strip() if code_context else ""
        input_vars = {id(var): name for name, var in frame.f_locals.items()}
        return line_of_code, input_vars

    def get_task(self, task_id):
        """Retrieve task entry based on the task ID."""
        task_entry = self.tasks.get(task_id)
        if not task_entry:
            raise ValueError(f"No task found for ID {task_id}")
        return task_entry

    def execute_task(self, task_id, inputs=None, area_id=None):
        if inputs is None:
            inputs = []
        
        logging.info(f"Executing task '{task_id}'...\n")
        
        frame = inspect.currentframe().f_back
        line_of_code, input_vars = self.get_caller_info(frame)
        input_var_names = [input_vars.get(id(inp), "N/A") for inp in inputs]
        task_entry = self.get_task(task_id)
        
        result = task_entry['task'](*inputs)
        
        # Extract the variable name to which the result is assigned
        output_match = re.search(r'^\s*([\w\s,]+?)\s*=', line_of_code)
        output_var_name = output_match.group(1).replace(" ", "") if output_match else "N/A"
        
        start_time = datetime.datetime.now()
        end_time = datetime.datetime.now()
        duration = (end_time - start_time).seconds
        
        # Log the task details internally
        self.tasks_log.append(task_id)
        self.tasks_details.append({
            'Time': start_time.strftime('%Y-%m-%d %H:%M:%S'),
            'Area ID': area_id,
            'Task ID': task_id,
            'Input': ", ".join(input_var_names),
            'Output': output_var_name,
            'Duration': f"{duration} seconds"
        })

        return result

    def show_lifecycle(self):
        """Display the model lifecycle details in a tabular format."""
        df = pd.DataFrame(self.tasks_details)
        return df



#### Register Developer Tasks

In [ ]:
# Instantiate the Developer class
developer = Developer()

# Set parameters
default_column = "default"

# Register developer tasks
developer.add_task(
    task_id="import_raw_data", 
    task=import_raw_data,
)

developer.add_task(
    task_id="add_default_definition",
    task=add_default_definition,  
)

developer.add_task(
    task_id="data_split",
    task=data_split, 
)

developer.add_task(
    task_id="drop_categories",
    task=drop_categories, 
)

developer.add_task(
    task_id="drop_features",
    task=drop_features, 
)

developer.add_task(
    task_id="convert_to_woe",
    task=convert_to_woe, 
)

developer.add_task(
    task_id="add_constant",
    task=add_constant, 
)

developer.add_task(
    task_id="train_model",
    task=train_model,
)

## Data Description

#### Import Raw Data: `df_1`

In [ ]:
lending_club_url = "https://vmai.s3.us-west-1.amazonaws.com/datasets/lending_club_loan_data_2007_2014.csv"

df_1 = developer.execute_task(
    area_id="data_description",
    task_id="import_raw_data", 
    inputs=[lending_club_url]   
)

#### Validate Dataset: `df_1`

In [ ]:
from validmind.vm_models.test_context import TestContext
from validmind.tests.data_validation.DescriptiveStatistics import DescriptiveStatistics

vm_df_1 = vm.init_dataset(dataset=df_1)
test_context_1 = TestContext(dataset=vm_df_1)

metric = DescriptiveStatistics(test_context_1)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.MissingValuesBarPlot import MissingValuesBarPlot

params = {"threshold": 80,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_1, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Preparation

#### Add Definition of Default: `df_2`

In [ ]:
df_2 = developer.execute_task(
    area_id="data_preparation",
    task_id="add_default_definition", 
    inputs=[df_1, default_column, unused_variables]
)

#### Validate Dataset: `df_2`

In [ ]:
vm_df = vm.init_dataset(
    dataset=df_2,
    target_column=default_column)

test_context_2 = TestContext(dataset=vm_df)

params = {"threshold": 80,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_2, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.ClassImbalance import ClassImbalance

metric = ClassImbalance(test_context_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.IQROutliersTable import IQROutliersTable

num_features = get_numerical_columns(df_2)
params = {"num_features": num_features,
          "threshold": 1.5
        }

metric = IQROutliersTable(test_context_2, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.IQROutliersBarPlot import IQROutliersBarPlot

num_features = get_numerical_columns(df_2)
params = {"num_features": num_features,
          "threshold": 1.5,
          "fig_width": 500}

metric = IQROutliersBarPlot(test_context_2, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Sampling

#### Sampling Method

We employ stratified sampling to create our training and testing sets. Stratified sampling is particularly important in this context. When the `stratify = y` parameter is set, it ensures that the distribution of the target variable (`y`) in the test set is the same as that in the original dataset. 

This is crucial for maintaining a consistent representation of the target variable classes, especially important in scenarios where the classes are imbalanced, which is often the case in credit risk scorecards.

#### Data Split: `df_train_2` and `df_test_2`

In [ ]:
df_train_2, df_test_2 = developer.execute_task(
    area_id="data_sampling",
    task_id="data_split", 
    inputs=[df_2, default_column]
)

## Exploratory Data Analysis 

#### Validate Dataset: `df_train_2`

In [ ]:
from validmind.tests.data_validation.TabularNumericalHistograms import TabularNumericalHistograms

vm_df_train_2 = vm.init_dataset(dataset=df_train_2,
                                target_column=default_column)
test_context_train_2 = TestContext(dataset=vm_df_train_2)

metric = TabularNumericalHistograms(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.HighCardinality import HighCardinality
metric = HighCardinality(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TabularCategoricalBarPlots import TabularCategoricalBarPlots
metric = TabularCategoricalBarPlots(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

#### Drop Categories: `df_train_3` and `df_test_3`

In [ ]:
df_train_3 = developer.execute_task(
    area_id="exploratory_data_analysis",
    task_id="drop_categories", 
    inputs=[df_train_2]
)

In [ ]:
df_test_3 = developer.execute_task(
    area_id="exploratory_data_analysis",
    task_id="drop_categories", 
    inputs=[df_test_2]
)

#### Validate Dataset: `df_train_3`

In [ ]:
from validmind.tests.data_validation.TargetRateBarPlots import TargetRateBarPlots

vm_df_train_3 = vm.init_dataset(
    dataset=df_train_3, 
    target_column=default_column)

test_context_train_3 = TestContext(dataset=vm_df_train_3)

# Configure the metric
params = {
    "default_column": default_column,
    "columns": None
}

metric = TargetRateBarPlots(test_context_train_3, params=params)
metric.run()
await metric.result.log()
metric.result.show()

#### Drop Features: `df_train_4` and `df_test_4`

In [ ]:
to_drop = ['addr_state', 'total_rec_int', 'loan_amnt',
                    'funded_amnt_inv', 'dti', 'revol_util', 'total_pymnt', 
                    'total_pymnt_inv', 'last_pymnt_amnt']

df_train_4 = developer.execute_task(
    area_id="exploratory_data_analysis",
    task_id="drop_features", 
    inputs=[df_train_3, to_drop]
)

In [ ]:
df_test_4 = developer.execute_task(
    area_id="exploratory_data_analysis",
    task_id="drop_features", 
    inputs=[df_test_3, to_drop]
)

#### Validate Dataset: `df_train_4`

In [ ]:
from validmind.tests.data_validation.ChiSquaredFeaturesTable import ChiSquaredFeaturesTable

vm_df_train_4 = vm.init_dataset(dataset=df_train_4, target_column=default_column)
test_context_train_4 = TestContext(dataset=vm_df_train_4)

cat_features = get_categorical_columns(df_train_4)
params = {"cat_features": cat_features,
          "p_threshold": 0.05}

metric = ChiSquaredFeaturesTable(test_context_train_4, params)
metric.run()
await metric.result.log() 
metric.result.show()

In [ ]:
from validmind.tests.data_validation.ANOVAOneWayTable import ANOVAOneWayTable

num_features = get_numerical_columns(df_train_4)
params = {"num_features": num_features,
          "p_threshold": 0.05}

metric = ANOVAOneWayTable(test_context_train_4, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.PearsonCorrelationMatrix import PearsonCorrelationMatrix

params = {"declutter": False,
          "features": None,
          "fontsize": 13}

metric = PearsonCorrelationMatrix(test_context_train_4, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.FeatureTargetCorrelationPlot import FeatureTargetCorrelationPlot

params = {"features": None}

metric = FeatureTargetCorrelationPlot(test_context_train_4, params)
metric.run()
await metric.result.log()
metric.result.show()

## Feature Engineering

#### Validate Dataset: `df_train_4`

In [ ]:
from validmind.tests.data_validation.WOEBinTable import WOEBinTable

metric = WOEBinTable(test_context_train_4)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_4, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.WOEBinPlots import WOEBinPlots

params = {
    "breaks_adj": {"int_rate": [5,10,15]},
    "fig_height": 500,
}

metric = WOEBinPlots(test_context_train_4, params=params)
metric.run()
await metric.result.log()
metric.result.show()

#### Convert Features into WoE Values: `df_train_5` and `df_test_5`

In [ ]:
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_4, params=params)
metric.run()
woe_dic = metric.result.metric.value['woe_iv']
woe_df = pd.DataFrame(woe_dic)

In [ ]:
df_train_5 = developer.execute_task(
    area_id="feature_engineering",
    task_id="convert_to_woe", 
    inputs=[df_train_4, woe_df, default_column]
)

In [ ]:
df_test_5 = developer.execute_task(
    area_id="feature_engineering",
    task_id="convert_to_woe", 
    inputs=[df_test_4, woe_df, default_column]
)

## Model Training

#### Add Constant: `df_train_6` and `df_test_6`

In [ ]:
df_train_6 = developer.execute_task(
    area_id="model_training",
    task_id="add_constant", 
    inputs=[df_train_5]
)

In [ ]:
df_test_6 = developer.execute_task(
    area_id="model_training",
    task_id="add_constant", 
    inputs=[df_test_5]
)

#### Train Model 1: `model_fit_1`

In [ ]:
model_fit_1 = developer.execute_task(
    area_id="model_training",
    task_id="train_model", 
    inputs=[df_train_6, default_column]
)

print(model_fit_1.summary())

#### Drop Features: `df_train_7` and `df_test_7`

In [ ]:
to_drop = []

df_train_7 = developer.execute_task(
    area_id="model_training",
    task_id="drop_features", 
    inputs=[df_train_6, to_drop]
)

In [ ]:
df_test_7 = developer.execute_task(
    area_id="model_training",
    task_id="drop_features", 
    inputs=[df_test_6, to_drop]
)

#### Train Model 2: `model_fit_2`

In [ ]:
model_fit_2 = developer.execute_task(
    area_id="model_training",
    task_id="train_model", 
    inputs=[df_train_7, default_column]
)

# Save the model and train dataset for PD development 
save_data = False
if save_data:
    save_model(model_fit_2, df=df_train_7, base_filename='model_fit_glm_scorecard')

print(model_fit_2.summary())

## Model Evaluation

#### Validate Model: `model_fit_2`

In [ ]:
# Create VM dataset
vm_df_train_7 = vm.init_dataset(
    dataset=df_train_7,
    target_column=default_column)
vm_df_test_7 = vm.init_dataset(
    dataset=df_test_7,
    target_column=default_column)

# Create VM model
vm_model_fit_2 = vm.init_model(
    model = model_fit_2, 
    train_ds=vm_df_train_7, 
    test_ds=vm_df_test_7)

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

test_context_models_fit_2 = TestContext(models = [vm_model_fit_2])

metric = RegressionCoeffsPlot(test_context_models_fit_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models_fit_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

test_context_model_fit_2 = TestContext(model= vm_model_fit_2)

# Configure test parameters
params = {
    "cut_off_threshold": 0.5,
}

metric = LogRegressionConfusionMatrix(test_context_model_fit_2, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionROCCurve import RegressionROCCurve

metric = RegressionROCCurve(test_context_model_fit_2)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.GINITable import GINITable

metric = GINITable(test_context_model_fit_2)
metric.run()
await metric.result.log() 
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogisticRegPredictionHistogram import LogisticRegPredictionHistogram

# Configure test parameters
params = {
    "title": "Histogram of Probability of Default",
}

metric = LogisticRegPredictionHistogram(test_context_model_fit_2, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogisticRegCumulativeProb import LogisticRegCumulativeProb

# Configure test parameters
params = {
    "title": "Cumulative Probability of Default",
}

metric = LogisticRegCumulativeProb(test_context_model_fit_2, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.ScorecardHistogram import ScorecardHistogram

# Configure test parameters
params = {
    "target_score": 600,
    "target_odds": 50,
    "pdo": 20,
    "title": "Histogram of Credit Scores",
}

metric = ScorecardHistogram(test_context_model_fit_2, params)
metric.run()
await metric.result.log()
metric.result.show()

#### Summary of Model Lifecycle

In [ ]:
lifecycle = developer.show_lifecycle()
display(lifecycle)